Mount drive and get csv file

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

data_string = "/content/drive/MyDrive/csc311/worded_data.csv"

import csv
data = csv.reader(open(data_string))


Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/csc311/worded_data.csv'

In [ ]:
import numpy as np


In [ ]:
vocab = None

vocab_set = set()
data = csv.reader(open(data_string))
for i, line in enumerate(data):
  if i == 0:
    continue
  review = [word for sublist in line[1:9] for word in sublist.split(",")]
  words = set(review)
  for w in words:
    vocab_set.add(w)

vocab = list(vocab_set)

print(vocab_set)

print("Vocabulary Size: ", len(vocab)) # Please include the output of this statement in your submission

{'', 'coffee', 'Weekend lunch', 'Horror', 'water', 'few', 'Weekend dinner', 'Music', 'simple', 'Week day lunch', 'many', 'very cheap', 'smoothie', 'very many', ' milkshake', 'Fantasy', 'other', ' smoothie', 'I will have some of this food item with my hot sauce', 'At a party', 'moderate', 'complex', 'very expensive', 'Sci-Fi', 'soda', 'Action', 'very complex', 'unknown', 'tea', 'Musical', 'Mystery', 'Week day dinner', 'Crime', 'Strangers', ' tea', 'Parents', 'Siblings', 'Late night snack', 'Thriller', 'Political', 'Comedy', 'Adventure', 'Teachers', 'Drama', 'Animation', 'Romance', 'alcohol', 'Family', ' juice', 'soup', 'asian alcohol', 'Friends', 'very simple', 'Documentary', ' asian alcohol', ' water', 'very few', 'asian pop', ' soda', 'milk', 'no_movie', 'cheap', 'A little (mild)', 'expensive', 'Superhero', 'juice', 'Sports', 'A lot (hot)', 'A moderate amount (medium)'}
Vocabulary Size:  69


Bag of words

In [ ]:
def make_bow(data, vocab):
    """
    Produce the bag-of-word representation of the data, along with a vector
    of labels. You *may* use loops to iterate over `data`. However, your code
    should not take more than O(len(data) * len(vocab)) to run.

    Parameters:
        `data`: a list of `(review, label)` pairs, like those produced from
                `list(csv.reader(open("trainvalid.csv")))`
        `vocab`: a list consisting of all unique words in the vocabulary

    Returns:
        `X`: A data matrix of bag-of-word features. This data matrix should be
             a numpy array with shape [len(data), len(vocab)].
             Moreover, `X[i,j] == 1` if the review in `data[i]` contains the
             word `vocab[j]`, and `X[i,j] == 0` otherwise.
        `t`: A numpy array of shape [len(data)], with `t[i] == 1` if
             `data[i]` is a positive review, and `t[i] == 0` otherwise.
    """
    X = np.zeros([len(data), len(vocab)])
    t = np.zeros([len(data)])
    vocab_dict = {word: idx for idx, word in enumerate(vocab)}
    print(vocab_dict)

    for i, row in enumerate(data):
        if row[9] == 'Pizza':
          t[i] = 0
        elif row[9] == 'Shawarma':
          t[i] = 1
        elif row[9] == "Sushi":
          t[i] = 2
        review = [word for sublist in row[1:9] for word in sublist.split(",")]

        print(review)
        words = set(review)
        for word in words:
            if word in vocab_dict:
                X[i, vocab_dict[word]] = 1
    return X, t




# Separate data into training/validation, then call `make_bow` to produce
# the bag-of-word features
import random
data = csv.reader(open(data_string))
random.seed(42)
data_list = list(data)
# print(data)
random.shuffle(data_list)
# print(data_list)

X_train, t_train = make_bow(data_list[:820], vocab)
X_valid, t_valid = make_bow(data_list[820:], vocab)
print(X_train)

{'': 0, 'coffee': 1, 'Weekend lunch': 2, 'Horror': 3, 'water': 4, 'few': 5, 'Weekend dinner': 6, 'Music': 7, 'simple': 8, 'Week day lunch': 9, 'many': 10, 'very cheap': 11, 'smoothie': 12, 'very many': 13, ' milkshake': 14, 'Fantasy': 15, 'other': 16, ' smoothie': 17, 'I will have some of this food item with my hot sauce': 18, 'At a party': 19, 'moderate': 20, 'complex': 21, 'very expensive': 22, 'Sci-Fi': 23, 'soda': 24, 'Action': 25, 'very complex': 26, 'unknown': 27, 'tea': 28, 'Musical': 29, 'Mystery': 30, 'Week day dinner': 31, 'Crime': 32, 'Strangers': 33, ' tea': 34, 'Parents': 35, 'Siblings': 36, 'Late night snack': 37, 'Thriller': 38, 'Political': 39, 'Comedy': 40, 'Adventure': 41, 'Teachers': 42, 'Drama': 43, 'Animation': 44, 'Romance': 45, 'alcohol': 46, 'Family': 47, ' juice': 48, 'soup': 49, 'asian alcohol': 50, 'Friends': 51, 'very simple': 52, 'Documentary': 53, ' asian alcohol': 54, ' water': 55, 'very few': 56, 'asian pop': 57, ' soda': 58, 'milk': 59, 'no_movie': 60, 

MAP - calculate Pi and Theta


In [ ]:
def naive_bayes_map(X, t):
    """
    Computer parameters pi and theta. adapted from lab to fit three targets
    """
    N, vocab_size = X.shape[0], X.shape[1]
    pi2 = (np.sum(t == 2) + 1) / (N + 2)
    pi1 = (np.sum(t == 1) + 1) / (N + 2)
    pi0 = (np.sum(t == 0) + 1) / (N + 2)
    theta = np.zeros([vocab_size, 3]) # TODO

    # these matrices may be useful (but what do they represent?)
    X_pizza = X[t == 0]
    X_shawarma = X[t == 1]
    X_sushi = X[t == 2]

    theta[:, 0] = (np.sum(X_pizza, axis=0) + 2 - 1) / (X_pizza.shape[0] + 2 + 2 -2) # TODO: you may uncomment this line if you'd like
    theta[:, 1] = (np.sum(X_shawarma, axis=0) + 2 - 1) / (X_shawarma.shape[0] + 2 + 2 -2)
    theta[:, 2] = (np.sum(X_sushi, axis=0) + 2 - 1) / (X_sushi.shape[0] + 2 + 2 -2)

    return pi0, pi1, pi2, theta

pi0, pi1, pi2, theta_map = naive_bayes_map(X_train, t_train)

print(theta_map.shape) # should be (549, 2)
print(pi0)
print(pi1)
print(pi2)

(69, 3)
0.35036496350364965
0.3223844282238443
0.3284671532846715


In [ ]:
def make_prediction(X, pi0, pi1, pi2, theta):
    """
    make prediction using pi and theta. Adapted from lab to fit three targets
    """
    N = X.shape[0]  # Number of samples
    y = np.zeros(N)  # Initialize predictions vector

    # Calculate log prior probabilities
    log_p0 = np.log(pi0)
    log_p1 = np.log(pi1)
    log_p2 = np.log(pi2)

    # Calculate log conditional probabilities for each class
    # Class 0 (Pizza)
    log_theta_0 = np.log(theta[:, 0])
    log_one_minus_theta_0 = np.log(1 - theta[:, 0])
    log_contrib_0 = np.dot(X, log_theta_0) + np.dot((1 - X), log_one_minus_theta_0)

    # Class 1 (Shawarma)
    log_theta_1 = np.log(theta[:, 1])
    log_one_minus_theta_1 = np.log(1 - theta[:, 1])
    log_contrib_1 = np.dot(X, log_theta_1) + np.dot((1 - X), log_one_minus_theta_1)

    # Class 2 (Sushi)
    log_theta_2 = np.log(theta[:, 2])
    log_one_minus_theta_2 = np.log(1 - theta[:, 2])
    log_contrib_2 = np.dot(X, log_theta_2) + np.dot((1 - X), log_one_minus_theta_2)

    # Calculate total log probabilities for each class
    log_p0_total = log_p0 + log_contrib_0
    log_p1_total = log_p1 + log_contrib_1
    log_p2_total = log_p2 + log_contrib_2

    # Stack log probabilities into a matrix [N, 3]
    log_probs = np.column_stack([log_p0_total, log_p1_total, log_p2_total])

    # For each sample, predict the class with highest probability
    y = np.argmax(log_probs, axis=1)

    return y

def accuracy(y, t):
    return np.mean(y == t)


y_map_train = make_prediction(X_train, pi0, pi1, pi2, theta_map)

y_map_valid = make_prediction(X_valid, pi0, pi1, pi2, theta_map)
print(y_map_valid)
print("MAP Train Acc:", accuracy(y_map_train, t_train))
print("MAP Valid Acc:", accuracy(y_map_valid, t_valid))

[2 0 1 1 2 1 2 0 1 0 0 0 0 2 1 2 2 1 1 1 1 1 2 1 0 1 1 0 2 2 0 2 2 1 1 1 1
 2 2 0 1 1 2 2 0 0 2 2 2 0 2 0 2 1 2 1 1 0 2 1 1 1 1 2 1 2 0 1 0 1 0 1 1 2
 0 0 1 1 2 2 1 0 0 1 0 0 1 1 2 1 1 0 2 2 1 1 0 1 0 0 0 2 1 1 2 2 1 0 2 1 2
 1 0 0 2 0 1 2 1 0 1 0 2 1 0 2 0 0 2 0 0 1 0 2 2 2 1 1 0 0 0 2 2 0 2 2 1 0
 0 0 1 2 0 0 2 1 2 0 0 1 0 1 1 1 2 1 0 1 1 1 2 0 1 2 2 1 2 1 0 1 2 2 1 2 0
 2 2 2 2 1 1 1 0 2 1 1 2 0 0 1 0 2 1 2 2 0 2 1 0 2 2 0 1 0 0 0 0 1 0 0 0 2
 0 0 0 2 1 2 1 1 2 0 2 2 2 1 2 1 0 2 1 2 2 2 2 0 0 1 0 0 1 0 2 1 0 0 1 2 1
 0 1 1 1 0 0 0 1 2 1 0 2 1 0 0 0 2 1 0 1 0 2 2 2 1 1 2 1 2 0 1 0 1 2 0 2 0
 0 1 2 2 2 2 2 0 2 2 1 0 2 1 2 2 2 0 2 1 2 1 1 2 0 0 0 1 2 0 1 1 2 2 2 1 1
 1 0 1 2 0 0 1 1 1 1 1 0 2 2 0 2 2 2 2 2 2 0 1 2 0 1 2 2 2 1 1 0 0 0 0 2 2
 2 0 2 1 0 2 1 1 2 1 2 1 0 0 2 1 0 1 0 2 0 0 0 2 0 0 1 0 1 1 2 2 1 1 0 1 2
 2 2 1 2 0 0 2 0 2 2 2 2 1 2 1 2 1 1 0 2 2 2 2 0 2 1 0 1 1 0 2 2 2 1 0 2 2
 0 2 1 2 1 1 1 1 1 0 0 1 1 2 0 0 1 0 2 1 2 1 2 2 1 2 2 1 2 2 0 0 1 1 0 2 2
 1 1 0 1 0 1 0 1 0 1 0 0 